In [369]:
import csv
import os
import sklearn
from sklearn import linear_model
import numpy as np
import pandas as pd
from random import *

In [3]:
df = pd.read_csv('./data/RegularSeasonDetailedResults.csv')                                              
team = pd.read_csv('./data/Teams.csv')              
winner = 'WTeamID'                                  
loser = 'LTeamID'                                   
start_year = 2003                                   
end_year = 2018                                     
min_team_id = df[winner].min()                      
max_team_id = df[winner].max()                      
df1 = df[['Season', winner, loser]].sort_values(by=['Season', winner, loser])                            
team_ids = np.sort(pd.concat([df1[winner], df1[loser]]).unique())                                        
win_mapping = {}                                    
loss_mapping = {}                                   
for i in range(start_year, end_year+1):             
    win_mapping[i] = pd.DataFrame(index=team_ids, columns=team_ids).fillna(0)                            
    loss_mapping[i] = pd.DataFrame(index=team_ids, columns=team_ids).fillna(0)                           
for _, row in df1.iterrows():                       
    win_mapping[row['Season']][row[winner]][row[loser]]+= 1                                              
    loss_mapping[row['Season']][row[loser]][row[winner]]+= 1                                             

In [15]:
wins = {}                                           
losses = {}                                         
percent_mapping = {}                                
percent_wins = {}                                   
for i in range(start_year, end_year+1):             
    i_wins = win_mapping[i]
    i_losses = loss_mapping[i]                      
    wins[i] = i_wins[list(i_wins.columns)].sum(axis=1)                                                   
    losses[i] = i_losses[list(i_losses.columns)].sum(axis=1)                                             
    percent_wins[i] = (wins[i]/(wins[i] + losses[i]) * 100).to_frame(i)                                  
percent_wins_by_year = pd.concat(percent_wins.values(), axis=1)                                          

In [73]:
tourn_res = pd.read_csv('./data/NCAATourneyDetailedResults.csv')                                         

In [342]:
seeds = pd.read_csv('./data/NCAATourneySeeds.csv')
seeds["Seed"] = seeds["Seed"].str[1:3]

In [332]:
win_mapping = {
'WTeamID': 'TeamID',                               
'WScore':  'Score',                                                                
'WFGM':    'FGM',                                  
'WFGA':    'FGA',                                  
'WFGM3':   'FGM3',                                 
'WFGA3':   'FGA3',                                 
'WFTM':    'FTM',                                  
'WFTA':    'FTA',                                  
'WOR':     'OR',                                   
'WDR':     'DR',                                  
'WAst':    'Ast',                                  
'WTO':     'TO',                                   
'WStl':    'Stl',                                  
'WBlk':    'Blk',                                  
'WPF':     'PF'}

loss_mapping = {                                    
'LTeamID': 'TeamID',                               
'LScore': 'Score',                                 
'LFGM': 'FGM',                                     
'LFGA': 'FGA',                           
'LFGM3': 'FGM3',                                    
'LFGA3': 'FGA3',                                   
'LFTM': 'FTM',                                      
'LFTA': 'FTA',                                      
'LOR': 'OR',                                        
'LDR': 'DR',                                        
'LAst': 'Ast',                                      
'LTO': 'TO',                                        
'LStl': 'Stl',                                      
'LBlk': 'Blk',                                      
'LPF': 'PF'}

win_drop = [
'LTeamID',                                         
'DayNum',
'LScore',                                           
'LFGM',                                             
'LFGA',                                             
'LFGM3',                                            
'LFGA3',                                            
'LFTM',                                             
'LFTA',                                             
'LOR',                                              
'LDR',                                              
'LAst',                                             
'LTO',                                              
'LStl',                                             
'LBlk',                                             
'LPF']

loss_drop = [
'WTeamID',
'DayNum',
'WScore',                                           
'WFGM',                                             
'WFGA',                                             
'WFGM3',                                            
'WFGA3',                                            
'WFTM',                                             
'WFTA',                                             
'WOR',                                              
'WDR',                                              
'WAst',                                             
'WTO',                                              
'WStl',                                             
'WBlk',                                             
'WPF']

stats_averages = {}

for i in range(start_year, end_year+1):
    year_data = df.loc[df['Season'] == i]
    w1 = year_data.groupby(['WTeamID']).sum()
    w2 = year_data.groupby(['WTeamID']).size().reset_index(name = 'Count')
    l1 = year_data.groupby(['LTeamID']).sum()
    l2 = year_data.groupby(['LTeamID']).size().reset_index(name = 'Count')

    loss = l2.join(l1, on='LTeamID').rename(columns=loss_mapping).drop(columns=loss_drop)
    win = w2.join(w1, on='WTeamID').rename(columns=win_mapping).drop(columns=win_drop)
    win[['IDX']] = win[['TeamID']]
    win = win.set_index('IDX')
    loss[['IDX']] = loss[['TeamID']]
    loss = loss.set_index('IDX')

    f = win.add(loss)
    
    f[['TeamID']] = f[['TeamID']] / 2
    f[['TeamID']] = f[['TeamID']].multiply(f['Count'], axis=0)
    f = f.divide(f['Count'], axis=0)
    f['FGR'] =  f['FGM']/f['FGA']
    f['FGR3'] = f['FGM3']/f['FGA3']
    f['FTR'] =  f['FTM']/f['FTA']
    stats_averages[i] = f

In [340]:
xTr = tourn_res[tourn_res.Season >= 2003]
xTr= tourn_res[['WTeamID','LTeamID','Season']]
xTr.insert(2, "Win", 1)
xTr.insert(0, "Team1", 1)
xTr.insert(1, "Team2", 1)

for index, row in xTr.iterrows():
    rand = random()
    if rand < .5:
        t1 = row["WTeamID"]
        t2 = row["LTeamID"]
        win = 1
    else:
        t1 = row["LTeamID"]
        t2 = row["WTeamID"]
        win = 0
    row["Team1"] = t1
    row["Team2"] = t2
    row["Win"] = win
xTr = xTr[["Team1", "Team2", 'Season', "Win"]]

In [547]:
def f(dictionary, index):                       
    d = {}
    for k,v in dictionary.items():              
        d[k] = {index: v}
    return d

g = lambda x: list(x.values())[0]

def merge(l):                                       
    fin = {}                                        
    for d in l:                                     
        for k,v in d.items():                       
            if k not in fin:                        
                fin[k] = v                          
            else:                                   
                fin[k].update(v)                    
    return fin

data_rows = []
for i,row in xTr.iterrows():
    try:
        season = row['Season']
        if (season == 2018):
            print('foo')
        stats = stats_averages[season]

        team1 = row['Team1']
        team2 = row['Team2']

        team1_stats = stats.loc[stats['TeamID'] == team1].rename(columns=lambda x: '1' + x)
        team2_stats = stats.loc[stats['TeamID'] == team2].rename(columns=lambda x: '2' + x)

        seed1 = int(seeds.loc[(seeds['TeamID'] == team1) & (seeds['Season'] == season)].iloc[0]['Seed'])
        seed2 = int(seeds.loc[(seeds['TeamID'] == team2) & (seeds['Season'] == season)].iloc[0]['Seed'])
        t1sd = team1_stats.to_dict()
        t2sd = team2_stats.to_dict()

        d = {k: g(v) for k, v in t1sd.items()}
        d.update({k: g(v) for k, v in t2sd.items()})
        d.update(xTr.ix[i].to_frame().T.iloc[0].to_dict())
        d.update({'seed1': seed1})
        d.update({'seed2': seed2})
        data_rows.append(f(d, i))
    except:
        continue

data = pd.DataFrame.from_dict(merge(data_rows)).drop(columns=['1Season', '2Season', '1TeamID', '2TeamID'])

In [560]:
train = data.loc[(data['Season'] >= 2003) & (data['Season'] < 2015)]
trainX = train.loc[:, data.columns != 'Win']
trainY = train['Win']
val = data.loc[(data['Season'] >= 2015) & (data['Season'] <= 2017)]
valX = val.loc[:, data.columns != 'Win']
valY = val['Win']
test = data.loc[(data['Season'] == 2017)]
testX = test.loc[:, data.columns != 'Win']
testY = test['Win']

In [377]:
classifier = linear_model.LogisticRegression('l2')

In [378]:
classifier.fit(trainX, trainY)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [379]:
classifier.score(valX, valY)

0.74626865671641796

In [380]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(trainX, trainY)

In [ ]:
from sklearn import neural_network
clf = neural_network.MLPClassifier()
clf = clf.fit(traunX, trainY)


In [601]:
matchup_rows = []
stats = stats_averages[2018]
season = 2018
i=0
for i1,row1 in seeds.loc[seeds['Season'] == 2018].iterrows():
    for i2,row2 in seeds.loc[seeds['Season'] == 2018].iterrows():
        try:
            team1 = row1['TeamID']
            team2 = row2['TeamID']
            
            if (team1 == team2) or (team1 > team2):
                continue
            team1_stats = stats.loc[stats['TeamID'] == team1].rename(columns=lambda x: '1' + x)
            team2_stats = stats.loc[stats['TeamID'] == team2].rename(columns=lambda x: '2' + x)

            seed1 = int(seeds.loc[(seeds['TeamID'] == team1) & (seeds['Season'] == season)].iloc[0]['Seed'])
            seed2 = int(seeds.loc[(seeds['TeamID'] == team2) & (seeds['Season'] == season)].iloc[0]['Seed'])
            
            t1sd = team1_stats.to_dict()
            t2sd = team2_stats.to_dict()
            
            d = {k: g(v) for k, v in t1sd.items()}
            d.update({k: g(v) for k, v in t2sd.items()})
            d.update({'Season': 2018})
            d.update({'Team1': team1})
            d.update({'Team2': team2})
            
            d.update({'seed1': seed1})
            d.update({'seed2': seed2})
            
            matchup_rows.append(f(d, i))
            i+=1
            
        except:
            continue

preds = pd.DataFrame.from_dict(merge(matchup_rows)).drop(columns=['1Season', '2Season', '1TeamID', '2TeamID'])


In [567]:
classifier = linear_model.LogisticRegression('l2', max_iter=1000)
classifier.fit(trainX, trainY)
classifier.score(valX, valY)


0.72448979591836737

In [578]:
probs = classifier.predict_proba(preds)

In [602]:
result = []
j = 0
for i in range(len(probs)):
    p = probs[i][1]
    team1 = str(preds['Team1'][i])
    team2 = str(preds['Team2'][i])
    string = '2018_' + team1 + '_' + team2
    seed1 = int(preds['seed1'][i])
    seed2 = int(preds['seed2'][i])
    team1name = team['TeamName'].loc[team['TeamID'] == int(team1)]
    team2name = team['TeamName'].loc[team['TeamID'] == int(team2)]
    if (p < 0.5) and (seed1 < seed2):
        print('{} upset {}\n\n'.format(team2name, team1name))
        j += 1
    result.append((string, p))

351    West Virginia
Name: TeamName, dtype: object upset 244    Purdue
Name: TeamName, dtype: object


351    West Virginia
Name: TeamName, dtype: object upset 302    Texas Tech
Name: TeamName, dtype: object


247    Rhode Island
Name: TeamName, dtype: object upset 95    Florida
Name: TeamName, dtype: object


38    Butler
Name: TeamName, dtype: object upset 3    Alabama
Name: TeamName, dtype: object


321    UNC Greensboro
Name: TeamName, dtype: object upset 281    St Bonaventure
Name: TeamName, dtype: object


321    UNC Greensboro
Name: TeamName, dtype: object upset 316    UCLA
Name: TeamName, dtype: object


271    SF Austin
Name: TeamName, dtype: object upset 166    Marshall
Name: TeamName, dtype: object


246    Radford
Name: TeamName, dtype: object upset 166    Marshall
Name: TeamName, dtype: object


184    Montana
Name: TeamName, dtype: object upset 166    Marshall
Name: TeamName, dtype: object


354    Wichita St
Name: TeamName, dtype: object upset 176    Michigan St
Name: Te

In [587]:
pd.DataFrame(result).to_csv('submission.csv')